In [ ]:
# import tracemalloc
# tracemalloc.start()
import pickle
from IPython.display import display
from rs_classes import async_request_client as async_client
import rs_functions.fetch_annotations_list as fetch_annotations
import rs_functions.fetch_annotation_content as fetch_annotation_content
import rs_functions.data_transformations as data_transformations
import rs_functions.fetch_annotation_meta as fetch_annotation_meta
import rs_functions.fetch_emails as fetch_emails

# #Initialize client
client = async_client.AsyncRequestClient("", "")

# Initialize the set_widgets list
url_input, bool_toggle, dropdown = data_transformations.create_input_widgets()
display(dropdown, url_input, bool_toggle)

In [3]:
token_input = ""
query = {
    "query": {
        "$and": [
            # {
            #     "queue": {
            #         "$in": [
            #             "https://elis.rossum.ai/api/v1/queues/XXXXXX",
            #             "https://elis.rossum.ai/api/v1/queues/XXXXXX",
            #             "https://elis.rossum.ai/api/v1/queues/XXXXXX"
            #         ]
            #     }
            # },
            # {
            #     "field.sender_name.string": {
            #         "$eq": "Contoso Ltd"
            #     }
            # },
            {
                "status": {
                    "$in": [
                        "confirmed",
                        "exported",
                        "to_review"
                    ]
                }
            },
            {
                "created_at": {"$gt": "2024-08-19T00:00:00"}
            }
        ]
    }
}
######################################################
if dropdown.label == "prod-eu2":
    url = f"https://{url_input.value}{dropdown.value}"
    client.reset_inputs(token_input, f"{url}/api")
else:
    url = f"{dropdown.value}"
    client.reset_inputs(token_input, f"{url}/api")
######################################################

In [ ]:
# to save data for later set True
save_data = False
saved_data_name = ""
load_from_archive = ""

######################################################
# Collect annotations based on search query
if load_from_archive:
    print("loading from archive")
    with open(f'data_archive/{load_from_archive}', 'rb') as file:
        annotations_collection = pickle.load(file)
else:        
    annotations_collection = await fetch_annotations.search_with_query(
            client, query, allPages=bool_toggle.value, page_max=None)

    await fetch_emails.get_email_content(client, annotations_collection)
